In [ ]:
!pip install mediapipe opencv-python pandas scikit-learn --user

## Importing required libraries.

In [1]:
import mediapipe as mp
import cv2

In [2]:
num_landmarks = 33 #number of landmarks used in the program

## Creating header row to the csv file.

In [3]:
landmarks = ['class']
for val in range(1, num_landmarks+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [4]:
landmarks

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

## Creating CSV file to save data extracted data

In [5]:
import csv

In [6]:
with open('data.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [7]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

## Function to process video to extract data and add them to the csv

In [8]:
def video_processor(class_name,video):
    cap = cv2.VideoCapture(video)
        # Initiate holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        while cap.isOpened():
            ret, frame = cap.read()
            if ret==False:
                break


            # Recolor Feed
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False        

            # Make Detections
            results = holistic.process(image)

            # Recolor image back to BGR for rendering
            image.flags.writeable = True   
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)


            # Pose Detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                     mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                     mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                     )
            # Export coordinates
            try:
                # Extract Pose landmarks
                pose = results.pose_landmarks.landmark
                pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten() if results.pose_landmarks else np.zeros(33*4))

                # Extract Face landmarks
#                 face = results.face_landmarks.landmark
#                 face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

                # Concate rows
#                 row = pose_row+face_row
                row = pose_row

                # Append class name 
                row.insert(0, class_name)

                # Export to CSV
                with open('data.csv', mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(row) 

            except:
                pass

            cv2.imshow('Raw incorrect h', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

In [9]:
import numpy as np
import glob

## Path to valid and invalid datasets

In [10]:
valid_path = "./datasets/valid/*.mp4"
invalid_path = "./datasets/invalid/*.mp4"

print("Valid video count : ", len(glob.glob(valid_path)))
print("Invalid video count :", len(glob.glob(invalid_path)))

Valid video count :  1
Invalid video count : 5


## Looping through valid and invalid datasets and extract data from them.

In [11]:
#adding invalid dataset values to the csv
class_name = "incorrect"
for i in range (1,len(glob.glob(invalid_path))+1):
    video_name = "./datasets/invalid/"+str(i)+".mp4"
    video_processor(class_name,video_name)
    print("passed "+str(i))

passed 1
passed 2
passed 3
passed 4
passed 5


In [12]:
#adding correct dataset values to the csv
class_name = "correct"
for i in range (1,len(glob.glob(valid_path))+1):
    video_name = "./datasets/valid/"+str(i)+".mp4"
    video_processor(class_name,video_name)
    print("passed "+str(i))

passed 1


## Visualising extracted data and Training data model

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [14]:
df = pd.read_csv('data.csv')

In [15]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,incorrect,0.515993,0.401074,-0.451521,0.999994,0.533684,0.389553,-0.404292,0.999992,0.544228,...,-0.417987,0.576736,0.616997,0.944889,-0.713074,0.527952,0.361875,0.924730,-0.643402,0.611416
1,incorrect,0.515971,0.401368,-0.673047,0.999994,0.533665,0.390346,-0.631668,0.999992,0.544149,...,-0.209274,0.591817,0.608270,0.934873,-0.473619,0.548130,0.363664,0.921075,-0.412065,0.630329
2,incorrect,0.515929,0.401593,-0.705014,0.999994,0.533657,0.390939,-0.665410,0.999992,0.544084,...,-0.197618,0.607700,0.596980,0.931402,-0.454663,0.564595,0.356700,0.920382,-0.400894,0.646520
3,incorrect,0.516267,0.402188,-0.721231,0.999994,0.533943,0.391689,-0.683475,0.999992,0.544140,...,-0.207068,0.625842,0.599120,0.928341,-0.447450,0.584130,0.355018,0.919099,-0.408734,0.665259
4,incorrect,0.516485,0.402322,-0.698062,0.999994,0.534263,0.391955,-0.660026,0.999992,0.544233,...,-0.222289,0.638456,0.598575,0.927167,-0.465927,0.596491,0.356780,0.919118,-0.426887,0.677319


In [16]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
16925,correct,0.494962,0.416560,-1.210146,0.999951,0.509524,0.390205,-1.178230,0.999967,0.520605,...,-0.747782,0.261256,0.793138,1.018255,-0.988007,0.228280,0.309480,1.019867,-1.133475,0.265662
16926,correct,0.497846,0.411333,-1.058305,0.999954,0.511724,0.386996,-1.015931,0.999969,0.522441,...,-0.759251,0.282393,0.794645,1.009841,-0.999944,0.237696,0.310593,1.011410,-1.142624,0.281154
16927,correct,0.500016,0.406482,-0.815641,0.999958,0.512796,0.384335,-0.770802,0.999970,0.523281,...,-0.900752,0.303723,0.794837,1.014610,-1.149668,0.252538,0.312189,1.012598,-1.276508,0.296409
16928,correct,0.500544,0.402046,-0.819248,0.999960,0.513299,0.382138,-0.771841,0.999972,0.523569,...,-0.906371,0.328586,0.794368,1.014891,-1.177943,0.271471,0.309683,1.010086,-1.287199,0.316980
16929,correct,0.504556,0.399228,-0.868992,0.999964,0.517284,0.380016,-0.817848,0.999974,0.526688,...,-0.854976,0.353701,0.792629,1.012745,-1.139979,0.290788,0.309005,1.008184,-1.243419,0.336925


In [17]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [19]:
y_test

8884       correct
8195       correct
3351     incorrect
11386      correct
4862       correct
           ...    
13002      correct
12149      correct
632      incorrect
3749     incorrect
10508      correct
Name: class, Length: 5079, dtype: object

In [20]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [25]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(max_iter=20000)),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [26]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [27]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression(max_iter=20000))]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [28]:
fit_models['rc'].predict(X_test)

array(['correct', 'correct', 'incorrect', ..., 'incorrect', 'incorrect',
       'correct'], dtype='<U9')

In [29]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [30]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9899586532782044
rc 0.9822799763733018
rf 1.0
gb 0.9998031108485922


In [31]:
fit_models['rf'].predict(X_test)

array(['correct', 'correct', 'incorrect', ..., 'incorrect', 'incorrect',
       'correct'], dtype=object)

In [32]:
y_test

8884       correct
8195       correct
3351     incorrect
11386      correct
4862       correct
           ...    
13002      correct
12149      correct
632      incorrect
3749     incorrect
10508      correct
Name: class, Length: 5079, dtype: object

## Saving best performing data model to a file

In [37]:
with open('shoulder_press.pkl', 'wb') as f:
    pickle.dump(fit_models['gb'], f)

In [38]:
with open('shoulder_press.pkl', 'rb') as f:
    model = pickle.load(f)

In [39]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gradientboostingclassifier', GradientBoostingClassifier())])

In [48]:
cap = cv2.VideoCapture('datasets/IMG_0126.MOV')
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if ret==False:
            break
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten()  if results.pose_landmarks else np.zeros(33*4))
 
            # Concate rows
            row = pose_row
            

            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X d

incorrect [0.05300758 0.94699242]
incorrect [0.05300758 0.94699242]
incorrect [0.05300758 0.94699242]


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X d

incorrect [0.05300758 0.94699242]
incorrect [0.03266865 0.96733135]
incorrect [0.03266865 0.96733135]


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X d

...

correct [0.99877646 0.00122354]
correct [0.99877646 0.00122354]
correct [0.99877646 0.00122354]


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X d

correct [0.99855245 0.00144755]
correct [0.99842569 0.00157431]
correct [0.99573279 0.00426721]
correct [0.99222211 0.00777789]
correct [0.99030587 0.00969413]


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:420: UserWarning: X d

KeyboardInterrupt: 

In [41]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))

(352, 129)